# Evaluating Chain-of-Thought and Code Interpreters

LLMs may utilize diverse generating strategies for complex queries.

We evaluate the performance of Chain-of-Thought, Code Interpreters and basic generation on the numerical reasoning tasks (with dataset FinHybrid).

In [1]:
# Preparation
import sys
import os
from pathlib import Path

# Get the project root directory
root_dir = Path(os.path.abspath("")).resolve().parents[1]
sys.path.append(str(root_dir))
# Change the working directory to the project root
os.chdir(root_dir)

res_dir = f"experiment/llm_gen/res/"
if not os.path.exists(res_dir):
    os.makedirs(res_dir)
    

In our paper, we use the powerful `text-embedding-3-large-model` model with AzureOpenAI-API . But you need to set up with your own api-key, endpoint and deploy-model in the config_file [uda/utils/access_config.py](../../uda/utils/access_config.py). 

If you want to use the API from **other alternative platforms** please change the codes in [uda/utils/retrieve.py (line-80)](../../uda/utils/retrieve.py#L81). 

For convenient demonstration, we choose the `BM25` retriever here.

In [2]:
# Experimental Configurations

# Available retrieval model_name: "bm25", "all-MiniLM-L6-v2", "all-mpnet-base-v2", "openai", "colbert"
# We choose bm25 for convenience
RT_MODEL = "bm25"

# This experiment is conducted solely on the FinHybrid dataset
DATASET_NAME = "fin"

LOCAL_LLM_DICT = {
    "llama-8B": "meta-llama/Meta-Llama-3-8B-Instruct",
    "code-llama-7b": "codellama/CodeLlama-7b-Instruct-hf",
    "code-llama-13b": "codellama/CodeLlama-13b-Instruct-hf",
}
LLM_LIST = ["gpt4", "llama-8B", "code-llama-7b", "code-llama-13b"]

LLM_LIST = LLM_LIST[:1]
DEMO_DOC_NUM = 2
DEMO_QA_NUM = 2

We use the **AzureOpenAI-API** serves as the interface for accessing GPT models. Users should set up the gpt-service with their own api-key and endpoint in the config_file [uda/utils/access_config.py](../../uda/utils/access_config.py). These configurations will be used in the `call_gpt()` function in the following codes.


If you want to use **other alternative platforms**, the `call_gpt()` can be replaced by the corresponding model-calling function.

In [3]:
from uda.utils import retrieve as rt
from uda.utils import retrieve_exp as rt_exp
from uda.utils import preprocess as pre
import pandas as pd
from uda.utils import llm
from uda.utils import inference
import json

# Strategies for LLM-generating 
strategeis = ["code", "cot", "basic"]

for LLM_MODEL in LLM_LIST:
    for strategy in strategeis:
        print(f"=== Start {DATASET_NAME} on {LLM_MODEL} with strategy: {strategy} ===")
        res_file = os.path.join(res_dir, f"{strategy}_{LLM_MODEL}_{RT_MODEL}.jsonl")

        # If use the local LLM, initialize the model
        if LLM_MODEL in LOCAL_LLM_DICT:
            llm_name = LOCAL_LLM_DICT[LLM_MODEL]
            llm_service = inference.LLM(llm_name)
            llm_service.init_llm()

        # Load the benchmark data
        bench_json_file = pre.meta_data[DATASET_NAME]["bench_json_file"]
        with open(bench_json_file, "r") as f:
            bench_data = json.load(f)

        # Run experiments on the demo docs
        doc_list = list(bench_data.keys())
        for doc in doc_list[:DEMO_DOC_NUM]:
            pdf_path = pre.get_example_pdf_path(DATASET_NAME, doc)
            if pdf_path is None:
                continue
            # Prepare the index for the document
            collection_name = f"{DATASET_NAME}_vector_db"
            collection = rt.prepare_collection(pdf_path, collection_name, RT_MODEL)
            for qa_item in bench_data[doc][:DEMO_QA_NUM]:
                question = qa_item["question"]
                # Retrieve the contexts
                contexts = rt.get_contexts(collection, question, RT_MODEL)
                context_text = '\n'.join(contexts)

                # Create the prompt for different generating strategies
                if strategy == "code":
                    llm_message = llm.make_prompt_code(question, context_text, DATASET_NAME, LLM_MODEL)
                elif strategy == "cot":
                    llm_message = llm.make_prompt(question, context_text, DATASET_NAME, LLM_MODEL)
                elif strategy == "basic":
                    llm_message = llm.make_prompt_basic(question, context_text, DATASET_NAME, LLM_MODEL)
                
                # Generate the answer
                if LLM_MODEL in LOCAL_LLM_DICT:
                    response = llm_service.infer(llm_message)
                elif LLM_MODEL == "gpt4":
                    # Set up with your own GPT4 service
                    response = llm.call_gpt(messages=llm_message)
                    if response is None:
                        print("Make sure your gpt4 service is set up correctly.")
                        raise Exception("GPT4 service")

                # log the results
                res_dict = {"model": LLM_MODEL, "question": question, "response": response, "doc": doc, "q_uid": qa_item["q_uid"], "answers": qa_item["answers"]}
                print(res_dict)
                with open(res_file, "a") as f:
                    f.write(json.dumps(res_dict) + "\n")
            rt.reset_collection(collection_name, RT_MODEL)
        print(f"=== Finish {strategy} on {LLM_MODEL} ===\n")


=== Start fin on gpt4 with strategy: code ===


2024-07-05 12:36:32,981 - INFO - HTTP Request: POST https://qinchuan-hui.openai.azure.com/openai/deployments/yulong-4/chat/completions?api-version=2024-04-01-preview "HTTP/1.1 200 OK"


{'model': 'gpt4', 'question': 'what percentage of total long-term assets under supervision are comprised of fixed income in 2015?', 'response': "To calculate the percentage of total long-term assets under supervision that are comprised of fixed income in 2015, we need to extract the relevant information from the context:\n\n- Fixed income assets under supervision in 2015: $546 billion\n- Total long-term assets under supervision in 2015: $946 billion\n\nNow we can calculate the percentage using the formula:\n\n\\( \\text{Percentage} = \\left( \\frac{\\text{Fixed Income Assets}}{\\text{Total Long-Term Assets}} \\right) \\times 100 \\)\n\nLet's do the calculation:\n\n```python\nfixed_income_assets_2015 = 546\ntotal_long_term_assets_2015 = 946\n\npercentage_fixed_income_2015 = (fixed_income_assets_2015 / total_long_term_assets_2015) * 100\nprint(percentage_fixed_income_2015)\n```", 'doc': 'GS_2016', 'q_uid': 'GS/2016/page_79.pdf-3', 'answers': {'str_answer': '57%', 'exe_answer': 0.57484}}


2024-07-05 12:36:40,450 - INFO - HTTP Request: POST https://qinchuan-hui.openai.azure.com/openai/deployments/yulong-4/chat/completions?api-version=2024-04-01-preview "HTTP/1.1 200 OK"


{'model': 'gpt4', 'question': 'what percentage of total long-term assets under supervision are comprised of fixed income in 2016?', 'response': "To calculate the percentage of total long-term assets under supervision that are comprised of fixed income in 2016, we need to use the following information from the context:\n\n- Fixed income assets under supervision in 2016: $601 billion\n- Total long-term assets under supervision in 2016: $1,021 billion\n\nNow, we can calculate the percentage using the formula:\n\n\\( \\text{Percentage} = \\left( \\frac{\\text{Fixed Income Assets}}{\\text{Total Long-Term Assets}} \\right) \\times 100 \\)\n\nLet's do the calculation:\n\n```python\nfixed_income_assets = 601\ntotal_long_term_assets = 1021\n\npercentage_fixed_income = (fixed_income_assets / total_long_term_assets) * 100\nprint(percentage_fixed_income)\n```", 'doc': 'GS_2016', 'q_uid': 'GS/2016/page_79.pdf-1', 'answers': {'str_answer': '59%', 'exe_answer': 0.588}}


...

{'model': 'gpt4', 'question': 'what percentage of total long-term assets under supervision are comprised of fixed income in 2016?', 'response': 'The answer is: 58.76%', 'doc': 'GS_2016', 'q_uid': 'GS/2016/page_79.pdf-1', 'answers': {'str_answer': '59%', 'exe_answer': 0.588}}


2024-07-05 12:37:30,355 - INFO - HTTP Request: POST https://qinchuan-hui.openai.azure.com/openai/deployments/yulong-4/chat/completions?api-version=2024-04-01-preview "HTTP/1.1 200 OK"


{'model': 'gpt4', 'question': 'what was the percentage change in the 5 year annual performance of the peer group stock from 2010 to 2011', 'response': 'The answer is: 36.78%.', 'doc': 'JKHY_2015', 'q_uid': 'JKHY/2015/page_20.pdf-2', 'answers': {'str_answer': '8.3%', 'exe_answer': 0.08276}}


2024-07-05 12:37:32,679 - INFO - HTTP Request: POST https://qinchuan-hui.openai.azure.com/openai/deployments/yulong-4/chat/completions?api-version=2024-04-01-preview "HTTP/1.1 200 OK"


{'model': 'gpt4', 'question': 'in 2010 , what was the cumulative total return of the s&p 500?', 'response': 'The answer is: 100.00', 'doc': 'JKHY_2015', 'q_uid': 'JKHY/2015/page_20.pdf-3', 'answers': {'str_answer': '30.69', 'exe_answer': 30.69}}
=== Finish basic on gpt4 ===



### Evaluate the results with different strategies

In [4]:
strategy="cot"
llm_model="gpt4"
rt_model="bm25"
res_file_name=f"experiment/llm_gen/res/{strategy}_{llm_model}_{rt_model}.jsonl"

from uda.eval.my_eval import eval_from_file
if strategy == "code":
    code_gen_flag = True
else:
    code_gen_flag = False
eval_from_file("fin", res_file_name, code_gen_flag)

Exact-match accuracy: 50.00
